In [1]:
!pip install sentence_transformers
!pip install pandas
!pip install openpyxl
!pip install datasets


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
! pip install -U accelerate
! pip install -U transformers


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from datasets import Dataset
from sentence_transformers import SentenceTransformer, losses, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
random.seed(42)

## 행민 데이터(2800개만)

In [4]:
df_public = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")[:2800]

In [5]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df_public.iterrows():
    # chunk = "\n".join([f"[{col}] {row[col]}" for col in chunks_in])
    chunk = f"'{row['사무명']}'은 {row['사무개요']} "
    documents.append(chunk)
print(documents[0])
df_public["chunk"] = documents

'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다. 


## GPT-4o mini QA생성 데이터

In [10]:
query_df = pd.read_excel("0_2799_질의응답데이터셋.xlsx")
query_df.head()

,사무명,질문,답변
0,토지임야대장등본교부 다량신청,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,안녕하세요! 여러 필지의 토지임야대장등본이 필요하시군요. 이 민원은 '토지임야대장등...
1,토지임야대장등본교부 다량신청,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...","토지임야대장등본을 한 번에 신청하고자 하신다면, 다음과 같은 정보를 참고하시면 됩니..."
2,토지임야대장등본교부 다량신청,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",상속받은 토지의 등본을 다량으로 신청하시는 것에 대해 안내해 드리겠습니다.\n\n먼...
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...","토지임야대장등본을 다량으로 신청하시려면, 인터넷으로 신청이 가능합니다. 이 민원은 ..."
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...","여러 필지의 토지임야대장등본을 한 번에 신청하시려는군요! 이 민원은 ""토지임야대장등..."


## QA와 행민 데이터 결합

In [11]:
# 병합 (Join)
df_merged = pd.merge(df_public, query_df, on='사무명', how='left')

print(df_merged.shape)
df_merged.head()

(14000, 18)


,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류,chunk,질문,답변
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,안녕하세요! 여러 필지의 토지임야대장등본이 필요하시군요. 이 민원은 '토지임야대장등...
1,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...","토지임야대장등본을 한 번에 신청하고자 하신다면, 다음과 같은 정보를 참고하시면 됩니..."
2,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",상속받은 토지의 등본을 다량으로 신청하시는 것에 대해 안내해 드리겠습니다.\n\n먼...
3,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...","토지임야대장등본을 다량으로 신청하시려면, 인터넷으로 신청이 가능합니다. 이 민원은 ..."
4,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...","여러 필지의 토지임야대장등본을 한 번에 신청하시려는군요! 이 민원은 ""토지임야대장등..."


In [14]:
chunks_in = ["사무명","질문", "chunk"]
df = df_merged[chunks_in]
df.head()

,사무명,질문,chunk
0,토지임야대장등본교부 다량신청,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
1,토지임야대장등본교부 다량신청,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
2,토지임야대장등본교부 다량신청,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...


In [26]:
train_df = df[df.index % 5 < 3]
test_df = df[df.index % 5 >= 3]
print(train_df.shape)
print(test_df.shape)
test_df.head()

(8400, 3)
(5600, 3)


,사무명,질문,chunk
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
8,공탁신청(공탁물지급청구 포함),4. 제가 대출금을 연체하여 채권자가 공탁을 요구하고 있습니다. 제가 공탁금을 지급...,"'공탁신청(공탁물지급청구 포함)'은 공탁 신청, 공탁금의 지급을 위한 민원입니다. ..."
9,공탁신청(공탁물지급청구 포함),5. 공탁금을 지급받으려면 특정한 조건이나 절차가 있는지 알고 싶습니다. 만약 저에...,"'공탁신청(공탁물지급청구 포함)'은 공탁 신청, 공탁금의 지급을 위한 민원입니다. ..."
13,친양자입양취소신고,"4. 친양자 입양을 취소할 때, 법원에서 어떤 기준으로 결정을 내리는지 궁금합니다....",'친양자입양취소신고'은 가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는...


### MultipleNegativesRankingLoss

In [27]:
train_examples = [
    InputExample(texts=[row['질문'], row['chunk']])
    for _, row in train_df.iterrows()
]

In [28]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=30,
    warmup_steps=100,
    output_path="./MNRLoss"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.039000
1000,0.013800
1500,0.013700
2000,0.012900
2500,0.009500
3000,0.009600
3500,0.008100
4000,0.008000
4500,0.009800
5000,0.008300
